In [1]:
import flask
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy

In [ ]:
# serve.py

from flask import Flask, redirect, url_for, request
from flask import render_template

engine = create_engine("sqlite:///project.db")
# creates a Flask application, named app
app = Flask(__name__)
counter=0

# a route where we will display a welcome message via an HTML template
@app.route("/")
def init():
    return render_template('table.html')

@app.route('/search')
def search():
    tableNames=engine.table_names()
    return render_template('search.html',tableNames=tableNames, message="Please Select Table", hidden2="hidden",hidden3="hidden",hiddenDelete="hidden")
@app.route('/insert')
def insert():
    tableNames=engine.table_names()
    return render_template('insert.html',tableNames=tableNames, message="Please Select Table", hidden2="hidden",hidden3="hidden",hiddenDelete="hidden")

    
@app.route('/predefined', methods = ['POST', 'GET'])
def select():
    if request.method == 'POST':
        x = request.form["query_nr"]
        return redirect(url_for('table', x=x))
    
@app.route('/searchBox', methods = ['POST', 'GET'])
def getInput():
    if request.method == 'POST':
        first = request.form["first"]
        second = request.form["second"]
        return redirect(url_for('searchResult', first=first,second=second))
@app.route('/insertForm', methods = ['POST', 'GET'])
def insertForm():
    global counter
    global first
    global tableColumns
    if request.method == 'POST':
        counter=counter+1
        if counter==1:
            first = request.form["tableInsert"]
            message="Fill in the values of the columns "+ first
            query="select * from "+first
            tableColumns=engine.execute(query).keys();
            
        if counter==2:
            query= "Insert into "+ first+ " Values ( ";
            for i in range(len(tableColumns)):
                query = query + " '"+request.form[tableColumns[i]]+"',"
            query = query[:-1]
            query=query +");"
            engine.execute(query) 
            counter=0
            query=""
            return render_template('insert.html', message="The values were inserted succesfuly",hidden1="hidden", hiddenSubmit="hidden", hidden2="hidden")    

        
        return render_template('insert.html', message=message, tableColumns=tableColumns,hidden1="hidden")    
@app.route('/testForm', methods = ['POST', 'GET'])
def testForm():
    global counter
    global first
    global second
    global third

    if request.method == 'POST':
        counter=counter+1
        if counter==1:
            first = request.form["testDD"]
            message="Please select one columns of the table "+ first
            query="select * from "+first
            columns=engine.execute(query).keys();
            return render_template('search.html',tableColumns=columns, hidden1="hidden",hidden3="hidden",hiddenDelete="hidden", message=message)
        if counter == 2:
            second = request.form["testDD2"]
            message="Please input the value of the column "+ second
            return render_template('search.html', hidden1="hidden",hidden2="hidden",hiddenDelete="hidden", message=message)
        if counter == 3:
            third=request.form["value"]
            counter=0
            return redirect(url_for('searchResult', first=first,second=second, third=third))

        
        return render_template('search.html', message=message)
@app.route('/delete', methods = ['POST', 'GET'])
def delete():
    if request.method == 'POST':
        query="Delete From "+first+" where "+second+" = "+"'"+third+"'"
        test = engine.execute(query)
        return render_template('search.html', hidden1="hidden",hidden2="hidden",hidden3="hidden",hiddenSubmit="hidden",hiddenDelete="hidden", message="Row(s) deleted")
    
@app.route("/searchResult/<first>/<second>/<third>")
def searchResult(first,second, third):
    query="Select * From "+first+" where "+second+" = "+"'"+third+"'"
    test = engine.execute(query).fetchall();
    columns=engine.execute(query).keys();
    count=len(engine.execute(query).keys());
    return render_template('search.html', test=test, columns=columns, count= count,hidden1="hidden",hidden3="hidden",hidden2="hidden",hiddenSubmit="hidden")
    

@app.route("/table/<x>")
def table(x):
    list_of_predefined = ["",
                          "Select L3.id from listing L3, provides p1 WHere L3.id=p1.id AND p1.amenities='wifi'", 
                          "SELECT AVG(review_scores_cleanliness) FROM Listing L, Amenities A, SCORE S, PROVIDE P WHERE L.id=S.id AND P.id=L.id AND P.amenities='TV' ;"
                         "SELECT DISTINCT L.host_id FROM Listing L, AVAILABLE_AT AV WHERE AV.id=L.id AND AV.available='t' AND AV.date <= DATE '2019-09-31' AND AV.date >= DATE '2019-03-00' ;"] 
    test = engine.execute(list_of_predefined[int(x)]).fetchall();
    columns=engine.execute(list_of_predefined[int(x)]).keys();
    count=len(engine.execute(list_of_predefined[int(x)]).keys());
    return render_template('table.html', test=test, columns=columns, count= count)
# run the application
if __name__=="__main__":
     app.run(port=5000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2019 15:57:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:33] "GET /insert HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:39] "POST /insertForm HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:42] "POST /insertForm HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:45] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:48] "POST /testForm HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:50] "POST /testForm HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:52] "POST /testForm HTTP/1.1" 302 -
127.0.0.1 - - [25/May/2019 15:57:53] "GET /searchResult/Country/country_code/Al HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:55] "POST /delete HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:57:57] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:58:00] "POST /testForm HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:58:01] "POST /testForm HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2019 15:58